# ECC

In [16]:
Pcurve = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 -1 # The proven prime
N=0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141 # Number of points in the field
Acurve = 0; Bcurve = 7 # These two defines the elliptic curve. y^2 = x^3 + Acurve * x + Bcurve
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
GPoint = (Gx,Gy) # This is our generator point. Trillions of dif ones possible
from sympy import mod_inverse

def modinv(a,n=Pcurve):
    return mod_inverse(a,n)
def ECadd(a,b): # Not true addition, invented for EC. Could have been called anything.
    LamAdd = ((b[1]-a[1]) * modinv(b[0]-a[0],Pcurve)) % Pcurve
    x = (LamAdd*LamAdd-a[0]-b[0]) % Pcurve
    y = (LamAdd*(a[0]-x)-a[1]) % Pcurve
    return (x,y)
def ECdouble(a): # This is called point doubling, also invented for EC.
    Lam = ((3*a[0]*a[0]+Acurve) * modinv((2*a[1]),Pcurve)) % Pcurve
    x = (Lam*Lam-2*a[0]) % Pcurve
    y = (Lam*(a[0]-x)-a[1]) % Pcurve
    Point = (x,y)
    return Point
def EccMultiply(GenPoint,ScalarHex): #Double & add. Not true multiplication
    if ScalarHex == 0 or ScalarHex >= N: 
        raise Exception("Invalid Scalar/Private Key")
    ScalarBin = str(bin(ScalarHex))[2:]
    Q=GenPoint
    for i in range (1, len(ScalarBin)): # This is invented EC multiplication.
        Q=ECdouble(Q);  #print ("DUB", Q[0])
        if ScalarBin[i] == "1":
            Q=ECadd(Q,GenPoint); #print ("ADD", Q[0])
    return (Q)

In [17]:
import random

ran = random.randrange(2**256)
myhex = "%064x" % ran
print(myhex)

0130d7bb226f49dc8288e78392f72827d077379fd73425e235102748221e69da


In [18]:
# Replace with any private key
privKey = 0x7f04c310135948617fcdaf5d90c918003b27e4d260eb11f045bffe527055d8f6

#Generate the public key
publicKey = EccMultiply(GPoint,privKey)
print ("the private key:")
print (hex(privKey))

the private key:
0x7f04c310135948617fcdaf5d90c918003b27e4d260eb11f045bffe527055d8f6


In [19]:
print ("the private key:")
print (hex(privKey))

print ("the public key x-value (Hex):")
print (hex(publicKey[0]))


print ("the public key y-value (Hex):")
print (hex(publicKey[1]))

the private key:
0x7f04c310135948617fcdaf5d90c918003b27e4d260eb11f045bffe527055d8f6
the public key x-value (Hex):
0xa3c157ae44d0a4041672964f0bb6b21af38cdf00e49f7e587e49140c77c9291f
the public key y-value (Hex):
0x9341fb5cad9c98b5f9cc6ae3f3ff0b9e4ebba42302fcdb551001ae303fe581bf


In [20]:
#Step 1 -> SHA2 Hash 
import hashlib

def encrypt_string(hash_string):
    sha_signature = \
        hashlib.sha256(hash_string.encode()).hexdigest()
    return sha_signature

hash_string = 'Quantum Computer'
sha_signature = encrypt_string(hash_string)
sha_signature_val = int(sha_signature,16)
print(hex(sha_signature_val))

0x50e4a8f3ab422ff40121e6416216c8938b8a464c0dc059ca2c6de96f1842124c


In [21]:
#Generate another random number
ran = random.randrange(2**256)
myhex = "%064x" % ran
k = int(myhex,16)
if(k>=1 and k<=N-1): 
    print("true")
print(hex(k))

true
0x92f8c64dbce5cdc7b41952976da7e5e38afe84dae985ab56b21c488ea831b820


In [22]:
#generate temporary public key(x1,y1)
x1,y1 = EccMultiply(GPoint,k)
print(hex(x1))
print(hex(y1))
r = x1 % N
s = (sha_signature_val+r*privKey) * modinv(k,N) % N
print(hex(r))
print(hex(s))

0xf344c6e261e81e5e9300fbc128625a2592d2c5ff4cc3df27d5d5ec671555eb47
0x7fd925149b1671b56b120b53ada78eb61734fd89fa501eafe55ccf52fc6a634c
0xf344c6e261e81e5e9300fbc128625a2592d2c5ff4cc3df27d5d5ec671555eb47
0x44ac24796919078c0b0ed31ce7efa82901376124f5312979d9f42f8269f5b190


In [23]:
#Verification Steps
if(r>N-1 or s>N-1): print("false")

w = modinv(s,N)
u1 = (sha_signature_val*w)%N
u2 = (r*w)%N
print(hex(u1))
print(hex(u2))
xu1 = EccMultiply(GPoint,u1)
xu2 = EccMultiply(publicKey,u2)
x2,y2 = ECadd(xu1,xu2)
print(hex(x2))
print(hex(y2))

print(r==x2)

0x11fcc785b29a2643d123eb3fa864d4719c605cd15a539cc68d1fac47b246181d
0xcd7deaddf04be5c497061ed63906f53cc524a0070c808be6f179a39cfd496ce2
0xf344c6e261e81e5e9300fbc128625a2592d2c5ff4cc3df27d5d5ec671555eb47
0x7fd925149b1671b56b120b53ada78eb61734fd89fa501eafe55ccf52fc6a634c
True
